In [1]:
import duckdb
import pandas as pd

In [8]:
meta = duckdb.sql("""
SELECT *
FROM '../data/processed/meta.parquet'""").to_df()

meta.head()

,appid,type,name,releasedate,freetoplay,genres,tags,languages,publishers,developers
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,21-Dec-17,0.0,"[Action, Adventure, Massively Multiplayer]","[Survival, Shooter, Multiplayer, PvP, FPS, Thi...","[English, Korean, Simplified Chinese, French, ...",[PUBG Corporation],[PUBG Corporation]
1,570,game,Dota 2,9-Jul-13,1.0,"[Action, Free to Play, Strategy]","[Free to Play, MOBA, Strategy, Multiplayer, Te...","[Bulgarian, Czech, Danish, Dutch, English, Fin...",[Valve],[Valve]
2,730,game,Counter-Strike: Global Offensive,21-Aug-12,1.0,"[Action, Free to Play]","[FPS, Multiplayer, Shooter, Action, Team-Based...","[Czech, Danish, Dutch, English, Finnish, Frenc...",[Valve],"[Valve, Hidden Path Entertainment]"
3,622590,None,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),None,NaN,[],[],[],[],[]
4,359550,game,Tom Clancy's Rainbow Six Siege,1-Dec-15,0.0,[Action],"[FPS, Multiplayer, Tactical, Action, Shooter, ...","[English, French, Italian, German, Spanish - S...",[Ubisoft],[Ubisoft Montreal]


In [3]:
serial_2017 = duckdb.sql("""
SELECT *
FROM '../data/processed/serial_2017.parquet'""").to_df()

serial_2017.head()

,datetime,active_users,appid
0,2017-12-14 00:00:00,8900.0,10
1,2017-12-14 00:05:00,8726.0,10
2,2017-12-14 00:10:00,8517.0,10
3,2017-12-14 00:15:00,8337.0,10
4,2017-12-14 00:20:00,8150.0,10


## Weekly Data
week, month, genres, weekly user, z-score, anomily, hhi

In [2]:
query = """
WITH hourly_max AS (
    SELECT
        DATE(datetime) AS day, appid, DATE_TRUNC('hour', datetime) AS hour_block,
        MAX(active_users) AS hour_user_count
    FROM (
        SELECT * FROM '../data/processed/serial_2017.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2018.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2019.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2020.parquet'
    )
    GROUP BY day, appid, hour_block
),

daily_sum AS (
    SELECT day, appid, SUM(hour_user_count) AS daily_user_count
    FROM hourly_max
    GROUP BY day, appid
),

weekly_avg AS (
    SELECT
        DATE_TRUNC('week', day) AS week_block,
        appid,
        AVG(daily_user_count) AS weekly_avg_user
    FROM daily_sum
    GROUP BY week_block, appid
)

SELECT w.week_block, w.appid, m.genres,
       SUM(w.weekly_avg_user) AS weekly_user,
FROM weekly_avg w
JOIN '../data/processed/meta.parquet' m
    ON w.appid = m.appid
GROUP BY w.week_block, w.appid, m.genres
ORDER BY w.week_block, w.appid, m.genres
"""

In [3]:
df_weekly_genre = duckdb.sql(query).to_df()

df_weekly_genre.head(10)


,week_block,appid,genres,weekly_user
0,2017-12-11,10,[Action],334741.50
1,2017-12-11,10090,[Action],21329.00
2,2017-12-11,10180,[Action],13752.00
3,2017-12-11,10190,[Action],31062.75
4,2017-12-11,10270,"[RPG, Strategy]",1032.75
5,2017-12-11,104900,"[Action, Adventure, Indie, RPG]",1178.00
6,2017-12-11,10500,[Strategy],69824.50
7,2017-12-11,105450,"[Simulation, Strategy]",44474.50
8,2017-12-11,105600,"[Action, Adventure, Indie, RPG]",456603.00
9,2017-12-11,107200,"[Action, Indie, RPG, Strategy]",898.25


In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# df_weekly_genre = ... (이미 쿼리로 불러온 DataFrame)

# 1. 장르별로 explode
df = df_weekly_genre.explode('genres')

# 2. 장르 목록
genre_list = sorted(df['genres'].dropna().unique())

# 4. 선택 장르 데이터
df_g = df.sort_values('week_block')
df_g = df_g[df_g['weekly_user'].notnull() & (df_g['weekly_user'] > 0)]

# 4. 날짜별로 weekly_user 합계
df_sum = df_g.groupby(['genres', 'week_block'], as_index=False)['weekly_user'].sum()

# 5. 변화율 및 z-score 계산
df_sum['weekly_user_pct'] = df_sum.groupby('genres')['weekly_user'].pct_change()

# 장르별로 expanding/rolling 계산
df_sum['exp_mean'] = df_sum.groupby('genres')['weekly_user_pct'].expanding(min_periods=3).mean().reset_index(level=0, drop=True)
df_sum['exp_std'] = df_sum.groupby('genres')['weekly_user_pct'].expanding(min_periods=3).std().replace(0, 1e-8).reset_index(level=0, drop=True)
df_sum['z_expanding'] = (df_sum['weekly_user_pct'] - df_sum['exp_mean']) / df_sum['exp_std']

df_sum['roll_mean'] = df_sum.groupby('genres')['weekly_user_pct'].rolling(window=26, min_periods=1).mean().reset_index(level=0, drop=True)
df_sum['roll_std'] = df_sum.groupby('genres')['weekly_user_pct'].rolling(window=26, min_periods=1).std().replace(0, 1e-8).reset_index(level=0, drop=True)
df_sum['z_rolling'] = (df_sum['weekly_user_pct'] - df_sum['roll_mean']) / df_sum['roll_std']

# index가 아니라, 각 장르별로 26개 미만은 expanding, 이상은 rolling 사용
def select_zscore(row):
    genre = row['genres']
    idx = row.name
    # 해당 장르의 row index
    genre_idx = df_sum[df_sum['genres'] == genre].index.get_loc(idx)
    if genre_idx < 26:
        return row['z_expanding']
    else:
        return row['z_rolling']

df_sum['zscore'] = df_sum.apply(select_zscore, axis=1)
df_sum['anomaly'] = df_sum['zscore'].abs() >= 1.8

In [5]:
df_sum.head()

,genres,week_block,weekly_user,weekly_user_pct,exp_mean,exp_std,z_expanding,roll_mean,roll_std,z_rolling,zscore,anomaly
0,Action,2017-12-11,8.342110e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,Action,2017-12-18,7.935139e+07,-0.048785,NaN,NaN,NaN,-0.048785,NaN,NaN,NaN,False
2,Action,2017-12-25,9.171796e+07,0.155846,NaN,NaN,NaN,0.053530,0.144696,0.707107,NaN,False
3,Action,2018-01-01,9.331822e+07,0.017448,0.041503,0.104415,-0.230380,0.041503,0.104415,-0.230380,-0.230380,False
4,Action,2018-01-08,8.169138e+07,-0.124593,-0.000021,0.119018,-1.046666,-0.000021,0.119018,-1.046666,-1.046666,False


In [6]:
import pandas as pd
import numpy as np

# 데이터 준비
df = df_weekly_genre.explode('genres').copy()
df['week_block'] = pd.to_datetime(df['week_block'])

# 주차별 전체 유저수
total_user_per_week = df.groupby('week_block')['weekly_user'].sum().rename('total_user')
df = df.merge(total_user_per_week, on='week_block')
df['share'] = df['weekly_user'] / df['total_user']

# 주차별 HHI 계산
hhi_weekly = (
    df.groupby('week_block')['share']
      .apply(lambda x: np.sum(x**2))
      .reset_index(name='hhi')
)

# 3개월(13주) rolling window로 HHI 평균
hhi_weekly['hhi_rolling'] = hhi_weekly['hhi'].rolling(window=13, min_periods=1).mean()

# 장르-주차별 HHI
genre_hhi = (
    df.groupby(['genres', 'week_block'])
      .apply(lambda x: np.sum((x['weekly_user'] / x['weekly_user'].sum())**2))
      .reset_index(name='hhi')
)
genre_hhi['hhi_rolling'] = genre_hhi.groupby('genres')['hhi'].transform(lambda x: x.rolling(window=13, min_periods=1).mean())

# 특정 장르 내 stacked bar용 데이터 (예: Action)
selected_genre = 'Action'
df_g = df[df['genres'] == selected_genre].copy()
df_g['weekly_user_rolling'] = df_g.groupby('appid')['weekly_user'].transform(lambda x: x.rolling(13, min_periods=1).sum())
bar_df = df_g.groupby(['week_block', 'appid'])['weekly_user_rolling'].sum().reset_index()

In [7]:
genre_hhi.head(10)

,genres,week_block,hhi,hhi_rolling
0,Action,2017-12-11,0.231672,0.231672
1,Action,2017-12-18,0.229277,0.230474
2,Action,2017-12-25,0.226590,0.229180
3,Action,2018-01-01,0.235147,0.230671
4,Action,2018-01-08,0.254511,0.235439
5,Action,2018-01-15,0.265814,0.240502
6,Action,2018-01-22,0.264240,0.243893
7,Action,2018-01-29,0.265155,0.246551
8,Action,2018-02-05,0.266458,0.248763
9,Action,2018-02-12,0.231699,0.247056


In [8]:
# df_sum와 genre_hhi를 genres, week_block 기준으로 조인하여 필요한 컬럼만 선택
weekly_df = pd.merge(
    df_sum,
    genre_hhi[['genres', 'week_block', 'hhi_rolling', 'hhi']],
    left_on=['genres', 'week_block'],
    right_on=['genres', 'week_block'],
    how='left'
)[['genres', 'week_block', 'weekly_user', 'zscore', 'anomaly', 'hhi_rolling', 'hhi']]

# week_block에서 month(yyyy-mm) 컬럼 추가
weekly_df['month'] = pd.to_datetime(weekly_df['week_block']).dt.strftime('%Y-%m')

weekly_df.head()

,genres,week_block,weekly_user,zscore,anomaly,hhi_rolling,hhi,month
0,Action,2017-12-11,8.342110e+07,NaN,False,0.231672,0.231672,2017-12
1,Action,2017-12-18,7.935139e+07,NaN,False,0.230474,0.229277,2017-12
2,Action,2017-12-25,9.171796e+07,NaN,False,0.229180,0.226590,2017-12
3,Action,2018-01-01,9.331822e+07,-0.230380,False,0.230671,0.235147,2018-01
4,Action,2018-01-08,8.169138e+07,-1.046666,False,0.235439,0.254511,2018-01


In [9]:
drop_g = ["Utilities", "Violent", "Design & Illustration", "Video Production", "Web Publishing", "Animation & Modeling", "Gore", "Nudity", "Sexual Content"]
weekly_df = weekly_df[~weekly_df['genres'].isin(drop_g)]
weekly_df.genres.unique()

array(['Action', 'Adventure', 'Casual', 'Early Access', 'Free to Play',
       'Indie', 'Massively Multiplayer', 'RPG', 'Racing', 'Simulation',
       'Sports', 'Strategy'], dtype=object)

In [11]:
weekly_df.to_parquet('../src/data/weekly_df.parquet')

In [12]:
import pandas as pd

df = pd.read_parquet('../src/data/weekly_df.parquet')

df.head(10)

,genres,week_block,weekly_user,zscore,anomaly,hhi_rolling,hhi,month
0,Action,2017-12-11,8.342110e+07,NaN,False,0.231672,0.231672,2017-12
1,Action,2017-12-18,7.935139e+07,NaN,False,0.230474,0.229277,2017-12
2,Action,2017-12-25,9.171796e+07,NaN,False,0.229180,0.226590,2017-12
3,Action,2018-01-01,9.331822e+07,-0.230380,False,0.230671,0.235147,2018-01
4,Action,2018-01-08,8.169138e+07,-1.046666,False,0.235439,0.254511,2018-01
5,Action,2018-01-15,8.586673e+07,0.387446,False,0.240502,0.265814,2018-01
6,Action,2018-01-22,8.492616e+07,-0.185945,False,0.243893,0.264240,2018-01
7,Action,2018-01-29,8.109644e+07,-0.500031,False,0.246551,0.265155,2018-01
8,Action,2018-02-05,5.561418e+07,-1.988176,True,0.248763,0.266458,2018-02
9,Action,2018-02-12,7.359468e+07,1.824444,True,0.247056,0.231699,2018-02


In [22]:
import plotly.graph_objs as go

fig = go.Figure()

for genre, group in df_sum.groupby('genres'):
    fig.add_trace(go.Scatter(
        x=group['week_block'],
        y=group['weekly_user'],
        mode='lines',
        name=genre,
        hoverinfo='text',
        text=[f"{genre}<br>주차: {w}<br>유저수: {u}" for w, u in zip(group['week_block'], group['weekly_user'])]
    ))

anomalies = df_sum[df_sum['anomaly']]
fig.add_trace(go.Scatter(
    x=anomalies['week_block'],
    y=anomalies['weekly_user'],
    mode='markers',
    name='이상값',
    marker=dict(color='red', size=10, symbol='circle-open'),
    text=[f"{row['genres']}<br>주차: {row['week_block']}<br>유저수: {row['weekly_user']}" for _, row in anomalies.iterrows()],
    hovertemplate='%{text}<extra></extra>',
    showlegend=True
))

fig.update_layout(
    title="장르별 주간 전체 유저수 및 이상값",
    xaxis_title="주차",
    yaxis_title="주간 전체 유저수",
    legend=dict(title='장르'),
    hovermode='closest'
)

fig.show()

## Monthly Meta
month, meta col, genres

In [13]:
query = """
WITH hourly_max AS (
    SELECT
        DATE(datetime) AS day, appid, DATE_TRUNC('hour', datetime) AS hour_block,
        MAX(active_users) AS hour_user_count
    FROM (
        SELECT * FROM '../data/processed/serial_2017.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2018.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2019.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2020.parquet'
    )
    GROUP BY day, appid, hour_block
),

daily_sum AS (
    SELECT day, appid, SUM(hour_user_count) AS daily_user_count
    FROM hourly_max
    GROUP BY day, appid
),

monthly_avg AS (
    SELECT
        DATE_TRUNC('month', day) AS month_block,
        appid,
        AVG(daily_user_count) AS monthly_avg_user
    FROM daily_sum
    GROUP BY month_block, appid
)

SELECT m.month_block, m.appid, meta.genres, meta.name, meta.developers, meta.languages,
       SUM(m.monthly_avg_user) AS monthly_user,
FROM monthly_avg m
JOIN '../data/processed/meta.parquet' meta
    ON m.appid = meta.appid
GROUP BY m.month_block, m.appid, meta.genres, meta.name, meta.developers, meta.languages
ORDER BY m.month_block, m.appid, meta.genres
"""

In [14]:
df_monthly_genre = duckdb.sql(query).to_df()

df_monthly_genre.head(10)


,month_block,appid,genres,name,developers,languages,monthly_user
0,2017-12-01,10,[Action],Counter-Strike,[Valve],"[English, French, German, Italian, Spanish - S...",354619.277778
1,2017-12-01,10090,[Action],Call of Duty: World at War,[Treyarch],"[English, French, Italian, German, Spanish - S...",25137.000000
2,2017-12-01,10180,[Action],Call of Duty: Modern Warfare 2,"[Infinity Ward, Aspyr (Mac)]","[English, French, German, Italian, Spanish - S...",15106.833333
3,2017-12-01,10190,[Action],Call of Duty: Modern Warfare 2 - Multiplayer,"[Infinity Ward, Aspyr (Mac)]","[English, French, German, Italian, Spanish - S...",33211.111111
4,2017-12-01,10270,"[RPG, Strategy]",Disciples III: Reincarnation,[Akella],"[English, Russian]",1119.111111
5,2017-12-01,104900,"[Action, Adventure, Indie, RPG]",ORION: Prelude,[Spiral Game Studios],[English],1336.222222
6,2017-12-01,10500,[Strategy],Empire: Total War,"[CREATIVE ASSEMBLY, Feral Interactive (Mac), F...","[English, French, Italian, German, Spanish - S...",72472.777778
7,2017-12-01,105450,"[Simulation, Strategy]",Age of Empires® III: Complete Collection,[Ensemble Studios],"[English, French, German, Italian, Spanish - S...",55902.888889
8,2017-12-01,105600,"[Action, Adventure, Indie, RPG]",Terraria,[Re-Logic],"[English, French, Italian, German, Spanish - S...",517424.500000
9,2017-12-01,107200,"[Action, Indie, RPG, Strategy]",Space Pirates and Zombies,[MinMax Games Ltd.],[English],948.000000


In [23]:
df = df_monthly_genre.explode('genres').copy()
df['month'] = pd.to_datetime(df['month_block']).dt.strftime('%Y-%m')
df.head()

,month_block,appid,genres,name,developers,languages,monthly_user,month
0,2017-12-01,10,Action,Counter-Strike,[Valve],"[English, French, German, Italian, Spanish - S...",354619.277778,2017-12
1,2017-12-01,10090,Action,Call of Duty: World at War,[Treyarch],"[English, French, Italian, German, Spanish - S...",25137.000000,2017-12
2,2017-12-01,10180,Action,Call of Duty: Modern Warfare 2,"[Infinity Ward, Aspyr (Mac)]","[English, French, German, Italian, Spanish - S...",15106.833333,2017-12
3,2017-12-01,10190,Action,Call of Duty: Modern Warfare 2 - Multiplayer,"[Infinity Ward, Aspyr (Mac)]","[English, French, German, Italian, Spanish - S...",33211.111111,2017-12
4,2017-12-01,10270,RPG,Disciples III: Reincarnation,[Akella],"[English, Russian]",1119.111111,2017-12


In [24]:
df.drop(columns=['month_block'], inplace=True)
df.head()

,appid,genres,name,developers,languages,monthly_user,month
0,10,Action,Counter-Strike,[Valve],"[English, French, German, Italian, Spanish - S...",354619.277778,2017-12
1,10090,Action,Call of Duty: World at War,[Treyarch],"[English, French, Italian, German, Spanish - S...",25137.000000,2017-12
2,10180,Action,Call of Duty: Modern Warfare 2,"[Infinity Ward, Aspyr (Mac)]","[English, French, German, Italian, Spanish - S...",15106.833333,2017-12
3,10190,Action,Call of Duty: Modern Warfare 2 - Multiplayer,"[Infinity Ward, Aspyr (Mac)]","[English, French, German, Italian, Spanish - S...",33211.111111,2017-12
4,10270,RPG,Disciples III: Reincarnation,[Akella],"[English, Russian]",1119.111111,2017-12


In [25]:
df = df[~df['genres'].isin(drop_g)]
df = df.dropna(subset=['genres'])
df.genres.unique()

array(['Action', 'RPG', 'Strategy', 'Adventure', 'Indie', 'Simulation',
       'Early Access', 'Free to Play', 'Massively Multiplayer', 'Racing',
       'Casual', 'Sports'], dtype=object)

In [26]:
df.to_parquet('../src/data/monthly_meta.parquet')

In [27]:
import pandas as pd

df = pd.read_parquet('../src/data/monthly_meta.parquet')

df.head(10)

,appid,genres,name,developers,languages,monthly_user,month
0,10,Action,Counter-Strike,[Valve],"[English, French, German, Italian, Spanish - S...",354619.277778,2017-12
1,10090,Action,Call of Duty: World at War,[Treyarch],"[English, French, Italian, German, Spanish - S...",25137.000000,2017-12
2,10180,Action,Call of Duty: Modern Warfare 2,"[Infinity Ward, Aspyr (Mac)]","[English, French, German, Italian, Spanish - S...",15106.833333,2017-12
3,10190,Action,Call of Duty: Modern Warfare 2 - Multiplayer,"[Infinity Ward, Aspyr (Mac)]","[English, French, German, Italian, Spanish - S...",33211.111111,2017-12
4,10270,RPG,Disciples III: Reincarnation,[Akella],"[English, Russian]",1119.111111,2017-12
4,10270,Strategy,Disciples III: Reincarnation,[Akella],"[English, Russian]",1119.111111,2017-12
5,104900,Action,ORION: Prelude,[Spiral Game Studios],[English],1336.222222,2017-12
5,104900,Adventure,ORION: Prelude,[Spiral Game Studios],[English],1336.222222,2017-12
5,104900,Indie,ORION: Prelude,[Spiral Game Studios],[English],1336.222222,2017-12
5,104900,RPG,ORION: Prelude,[Spiral Game Studios],[English],1336.222222,2017-12


## monthly weekday
month, weekday, genre, active user

In [28]:
query = """
WITH hourly_max AS (
    SELECT
        DATE(datetime) AS day, appid, DATE_TRUNC('hour', datetime) AS hour_block,
        MAX(active_users) AS hour_user_count
    FROM (
        SELECT * FROM '../data/processed/serial_2017.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2018.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2019.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2020.parquet'
    )
    GROUP BY appid, hour_block, day
),

weekday_avg AS (
    SELECT EXTRACT(dow FROM hour_block) AS weekday,
           STRFTIME(hour_block, '%Y-%m') AS month,
           appid, AVG(hour_user_count) AS weekday_user_count
    FROM hourly_max
    GROUP BY weekday, month, appid
)

SELECT w.weekday, w.month, meta.genres, w.appid,
       AVG(w.weekday_user_count) AS weekday_user_count
FROM weekday_avg w
JOIN '../data/processed/meta.parquet' meta
    ON w.appid = meta.appid
GROUP BY w.weekday, w.month, meta.genres, w.appid
ORDER BY  w.weekday, w.month
"""

In [29]:
df_weekday_genre = duckdb.sql(query).to_df()

df_weekday_genre.head(10)


,weekday,month,genres,appid,weekday_user_count
0,0,2017-12,[Simulation],314160,3153.777778
1,0,2017-12,"[Casual, Indie]",454080,26.638889
2,0,2017-12,"[Action, Adventure, Casual, Indie, Simulation]",384190,77.611111
3,0,2017-12,"[Action, Adventure]",368500,615.680556
4,0,2017-12,"[Action, Adventure, Indie]",312520,80.041667
5,0,2017-12,"[Indie, RPG, Simulation, Strategy]",718650,31.333333
6,0,2017-12,"[Simulation, Strategy]",493340,4016.750000
7,0,2017-12,"[Indie, RPG, Strategy]",248860,73.055556
8,0,2017-12,"[Free to Play, Massively Multiplayer, RPG]",39120,477.027778
9,0,2017-12,"[Action, Adventure, Casual, Massively Multipla...",454650,2470.597222


In [30]:
df = df_weekday_genre.explode('genres').copy()
# 1. 장르별로 groupby를 하고, weekday와 month가 같은 것들끼리 weekday_user_count를 합산합니다.
# 2. 결과를 새로운 DataFrame에 저장합니다.

df_grouped = (
    df.groupby(['genres', 'weekday', 'month'], as_index=False)['weekday_user_count']
      .sum()
)

# 결과 확인 (상위 10개)
df_grouped.head(10)


,genres,weekday,month,weekday_user_count
0,Action,0,2017-12,3.813086e+06
1,Action,0,2018-01,4.047632e+06
2,Action,0,2018-02,3.618018e+06
3,Action,0,2018-03,3.588510e+06
4,Action,0,2018-04,3.139800e+06
5,Action,0,2018-05,2.890227e+06
6,Action,0,2018-06,2.887413e+06
7,Action,0,2018-07,2.707005e+06
8,Action,0,2018-08,2.590527e+06
9,Action,0,2018-09,2.571702e+06


In [33]:
df_grouped = df_grouped[~df_grouped['genres'].isin(drop_g)]
df_grouped = df_grouped.dropna(subset=['genres'])
df_grouped.genres.unique()

array(['Action', 'Adventure', 'Casual', 'Early Access', 'Free to Play',
       'Indie', 'Massively Multiplayer', 'RPG', 'Racing', 'Simulation',
       'Sports', 'Strategy'], dtype=object)

In [34]:
df_grouped.to_parquet('../src/data/monthly_weekday.parquet')

## monthly hour
month, hour, genre, active user

In [35]:
query = """
WITH hourly_max AS (
    SELECT
        DATE(datetime) AS day, appid, DATE_TRUNC('hour', datetime) AS hour_block,
        MAX(active_users) AS hour_user_count
    FROM (
        SELECT * FROM '../data/processed/serial_2017.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2018.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2019.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2020.parquet'
    )
    GROUP BY appid, hour_block, day
),

hour_avg AS (
    SELECT EXTRACT(hour FROM hour_block) AS hour,
           STRFTIME(hour_block, '%Y-%m') AS month,
           appid, AVG(hour_user_count) AS hour_user_count
    FROM hourly_max
    GROUP BY hour, month, appid
)

SELECT w.hour, w.month, meta.genres, w.appid,
       AVG(w.hour_user_count) AS hour_user_count
FROM hour_avg w
JOIN '../data/processed/meta.parquet' meta
    ON w.appid = meta.appid
GROUP BY w.hour, w.month, meta.genres, w.appid
ORDER BY  w.hour, w.month
"""

In [36]:
df_hour_genre = duckdb.sql(query).to_df()

df_hour_genre.head(10)


,hour,month,genres,appid,hour_user_count
0,0,2017-12,"[Action, Adventure, Free to Play, Indie, Massi...",238960,41241.833333
1,0,2017-12,[],295270,2346.555556
2,0,2017-12,"[Free to Play, Massively Multiplayer, Strategy]",339600,954.111111
3,0,2017-12,"[Casual, Indie, Simulation]",517160,45.166667
4,0,2017-12,"[Action, Adventure, Indie, RPG]",268750,76.222222
5,0,2017-12,[Racing],378860,1105.944444
6,0,2017-12,[],657860,24.166667
7,0,2017-12,[],19930,42.166667
8,0,2017-12,"[Indie, RPG, Strategy]",248860,74.222222
9,0,2017-12,"[Action, Adventure, Indie]",95400,45.111111


In [37]:
df = df_hour_genre.explode('genres').copy()
# 1. 장르별로 groupby를 하고, weekday와 month가 같은 것들끼리 weekday_user_count를 합산합니다.
# 2. 결과를 새로운 DataFrame에 저장합니다.

df_grouped = (
    df.groupby(['genres', 'hour', 'month'], as_index=False)['hour_user_count']
      .sum()
)

# 결과 확인 (상위 10개)
df_grouped.head(10)


,genres,hour,month,hour_user_count
0,Action,0,2017-12,2.124914e+06
1,Action,0,2018-01,2.120065e+06
2,Action,0,2018-02,1.797059e+06
3,Action,0,2018-03,1.732240e+06
4,Action,0,2018-04,1.520133e+06
5,Action,0,2018-05,1.440250e+06
6,Action,0,2018-06,1.534700e+06
7,Action,0,2018-07,1.578882e+06
8,Action,0,2018-08,1.490445e+06
9,Action,0,2018-09,1.395881e+06


In [38]:
df_grouped = df_grouped[~df_grouped['genres'].isin(drop_g)]
df_grouped = df_grouped.dropna(subset=['genres'])
df_grouped.genres.unique()

array(['Action', 'Adventure', 'Casual', 'Early Access', 'Free to Play',
       'Indie', 'Massively Multiplayer', 'RPG', 'Racing', 'Simulation',
       'Sports', 'Strategy'], dtype=object)

In [39]:
df_grouped.to_parquet('../src/data/monthly_hour.parquet')

## monthly weekday - hour by genre
for every genre
month, weekday, hour, active user

In [40]:
query = """
WITH hourly_max AS (
    SELECT
        DATE(datetime) AS day, appid, DATE_TRUNC('hour', datetime) AS hour_block,
        MAX(active_users) AS hour_user_count
    FROM (
        SELECT * FROM '../data/processed/serial_2017.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2018.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2019.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2020.parquet'
    )
    GROUP BY appid, hour_block, day
),

weekday_avg AS (
    SELECT EXTRACT(dow FROM hour_block) AS weekday,
           EXTRACT(hour FROM hour_block) as hour,
           STRFTIME(hour_block, '%Y-%m') AS month,
           appid, AVG(hour_user_count) AS weekday_user_count, day
    FROM hourly_max
    GROUP BY weekday, hour, appid, day, month
)

SELECT w.weekday, w.hour, w.month, w.appid, meta.genres,
       AVG(w.weekday_user_count) AS weekday_user_count
FROM weekday_avg w
JOIN '../data/processed/meta.parquet' meta
    ON w.appid = meta.appid
GROUP BY w.month, w.weekday, w.hour, w.appid, meta.genres
ORDER BY  w.weekday, w.hour, w.appid
"""

In [41]:
df_wday_hour_genre = duckdb.sql(query).to_df()

df_wday_hour_genre.head(10)


,weekday,hour,month,appid,genres,weekday_user_count
0,0,0,2019-01,10,[Action],11156.00
1,0,0,2020-07,10,[Action],10557.00
2,0,0,2019-04,10,[Action],7504.75
3,0,0,2018-07,10,[Action],8649.80
4,0,0,2020-01,10,[Action],9342.00
5,0,0,2018-03,10,[Action],11797.75
6,0,0,2018-04,10,[Action],9087.80
7,0,0,2019-05,10,[Action],7500.50
8,0,0,2018-02,10,[Action],12674.00
9,0,0,2020-08,10,[Action],9618.50


In [42]:
df = df_wday_hour_genre.explode('genres').copy()
df = df[~df['genres'].isin(drop_g)]
df = df.dropna(subset=['genres'])
for g in df['genres'].unique():
    print(f'processing {g}')
    df_g = df[df['genres'] == g]
    df_g = df_g.groupby(['month','weekday', 'hour'])['weekday_user_count'].sum().reset_index()
    df_g.to_parquet(f'../src/data/wday_hour_{g}.parquet')

processing Action
processing RPG
processing Strategy
processing Adventure
processing Indie
processing Simulation
processing Early Access
processing Free to Play
processing Massively Multiplayer
processing Racing
processing Casual
processing Sports


In [2]:
query = """
WITH hourly_max AS (
    SELECT
        DATE(datetime) AS day, appid, DATE_TRUNC('hour', datetime) AS hour_block,
        MAX(active_users) AS hour_user_count
    FROM (
        SELECT * FROM '../data/processed/serial_2017.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2018.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2019.parquet'
        UNION ALL
        SELECT * FROM '../data/processed/serial_2020.parquet'
    )
    GROUP BY day, appid, hour_block
),

daily_sum AS (
    SELECT day, appid, SUM(hour_user_count) AS daily_user_count
    FROM hourly_max
    GROUP BY day, appid
),

monthly_avg AS (
    SELECT
        DATE_TRUNC('month', day) AS month_block,
        appid,
        AVG(daily_user_count) AS monthly_avg_user
    FROM daily_sum
    GROUP BY month_block, appid
)

SELECT m.month_block, m.appid, meta.genres, meta.name,
       SUM(m.monthly_avg_user) AS monthly_user
FROM monthly_avg m
JOIN '../data/processed/meta.parquet' meta
    ON m.appid = meta.appid
GROUP BY m.month_block, m.appid, meta.genres, meta.name
ORDER BY m.month_block, m.appid, meta.genres
"""

In [3]:
df_monthly_genre = duckdb.sql(query).to_df()

df_monthly_genre.head(10)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,month_block,appid,genres,name,monthly_user
0,2017-12-01,10,[Action],Counter-Strike,354619.277778
1,2017-12-01,10090,[Action],Call of Duty: World at War,25137.000000
2,2017-12-01,10180,[Action],Call of Duty: Modern Warfare 2,15106.833333
3,2017-12-01,10190,[Action],Call of Duty: Modern Warfare 2 - Multiplayer,33211.111111
4,2017-12-01,10270,"[RPG, Strategy]",Disciples III: Reincarnation,1119.111111
5,2017-12-01,104900,"[Action, Adventure, Indie, RPG]",ORION: Prelude,1336.222222
6,2017-12-01,10500,[Strategy],Empire: Total War,72472.777778
7,2017-12-01,105450,"[Simulation, Strategy]",Age of Empires® III: Complete Collection,55902.888889
8,2017-12-01,105600,"[Action, Adventure, Indie, RPG]",Terraria,517424.500000
9,2017-12-01,107200,"[Action, Indie, RPG, Strategy]",Space Pirates and Zombies,948.000000


In [4]:
df = df_monthly_genre.explode('genres').copy()

drop_g = ["Utilities", "Violent", "Design & Illustration", "Video Production", "Web Publishing", "Animation & Modeling", "Gore", "Nudity", "Sexual Content"]
df = df[~df['genres'].isin(drop_g)]
df.head()

,month_block,appid,genres,name,monthly_user
0,2017-12-01,10,Action,Counter-Strike,354619.277778
1,2017-12-01,10090,Action,Call of Duty: World at War,25137.000000
2,2017-12-01,10180,Action,Call of Duty: Modern Warfare 2,15106.833333
3,2017-12-01,10190,Action,Call of Duty: Modern Warfare 2 - Multiplayer,33211.111111
4,2017-12-01,10270,RPG,Disciples III: Reincarnation,1119.111111


In [14]:
df_top5 = (df.groupby(['genres', 'month_block'], group_keys=False)
             .apply(lambda g : pd.concat([
                 g.nlargest(5, 'monthly_user'),
                 pd.DataFrame({
                     'genres': [g.genres.iloc[0]],
                     'month_block': [g.month_block.iloc[0]],
                     'monthly_user': [g['monthly_user'].iloc[5:].sum()],
                     'appid': [None],
                     'name': ['Others']
                 })
             ]) if len(g) > 5 else g)
             .reset_index(drop=True)
)

df_top5.head()

C:\Users\변자민\AppData\Local\Temp\ipykernel_13640\4243603223.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g : pd.concat([


,month_block,appid,genres,name,monthly_user
0,2017-12-01,578080,Action,PLAYERUNKNOWN'S BATTLEGROUNDS,3.724404e+07
1,2017-12-01,570,Action,Dota 2,1.310847e+07
2,2017-12-01,730,Action,Counter-Strike: Global Offensive,8.702029e+06
3,2017-12-01,271590,Action,Grand Theft Auto V,2.407840e+06
4,2017-12-01,359550,Action,Tom Clancy's Rainbow Six Siege,1.498630e+06


In [15]:
df_top5.to_parquet('../src/data/top5_monthly_genre.parquet')